## Estructura de los Datos

### Cash_Request (CR)

* **CR.id**:
  Unique ID of Cash Request.

* **CR.user_id**:
  Unique ID of the user who requested the cash advance.

* **CR.deleted_account_id**:
  If a user delete his account, we are replacing the user_id by this id. 
  
  It corresponds to a unique ID in the deleted account table with some keys information saved for fraud-fighting purposes (while respecting GDPR regulation).

* **CR.amount**:
  Amount of the Cash Request.

* **CR.created_at**:
  Timestamp of the CR creation.

* **CR.updated_at**:
  Timestamp of the latest CR's details update **(= update of at least one column in this table)**.

* **CR.reimbursement_date**:
  Planned reimbursement date. The user card will be charged at this date.

* **CR.cash_request_received_date**:
  Date of the receipt of the CR. Based on user's bank history.

* **CR.money_back_date**:
  Date where the CR was considered as money back. 
  
  It's either the paid_by_card date or the date were we considered that's the direc debit have low odds to be rejected (based on business rules) 

* **CR.send_at**:
  Timestamp of the funds's transfer.

* **CR.transfer_type**:
  Possible values are:

  | Campo        | Registros | Descripcion                                                  |
  | ------------ | --------: | ------------------------------------------------------------ |
  | **instant**: |     13882 | El usuario eligió recibir el adelanto instantáneamente. (user choose not received the advance instantly) |
  | **regular**: |     10086 | El usuario eligió no pagar inmediatamente y esperar la transferencia. (user choose to not pay and wait for the transfer) |

---

* **CR.reco_creation**:
  Timestamp of the recovery creation.
* **CR.reco_last_update**:
  Timestamp of the last recovery case update. Can be used to determine the incident closure date.
* **CR.moderated_at**:
  Timestamp of the manual review. Only filled if the CR needed a manual review.

* **CR.recovery_status**:
  Possible values are:

  | Campo                     | Registros | Descripcion                                                  |
  | ------------------------- | --------: | ------------------------------------------------------------ |
  | **NULL (Nice)**:          |     20639 | El CR nunca tuvo un incidente de pago. (Null if the cash request never had a payment incident.) |
  | **completed**:            |      2467 | El incidente de pago fue resuelto (el CR fue reembolsado). (the payment incident was resolved **(=the cash request was reimbursed)**) |
  | **pending**: (+-=direct_debit_rejected): |       845 | El incidente de pago aún está abierto. (the payment incident still open) |
  | **pending_direct_debit**: |        16 | El incidente de pago sigue abierto, pero se ha lanzado un débito directo SEPA. (the payment incident still open but a SEPA direct debit is launched) |
  | **cancelled**:            |         1 | ?? No figura a la documentacio                               |


* **CR.status**:     (23970 registros)
  Status of the CR. Possible values are: 

  | Campo                                                     | Registros | Descripcion                                                  |
  | --------------------------------------------------------- | --------: | ------------------------------------------------------------ |
  | **money_back**:                                           |     16395 | El CR fue reembolsado exitosamente.(The CR was successfully reimbursed.) |
  | **active**:                                               |        59 | Los fondos fueron recibidos en la cuenta del cliente. (Funds were received on the customer account.) |
  | **direct_debit_sent**:                                    |        34 | Se envió un débito directo SEPA, pero aún no se confirma el resultado. (We sent/scheduled a SEPA direct debit to charge the customer account. The result of this debit is not yet confirmed) |
  |                                                           |           |                                                              |
  | **rejected**:                                             |      6568 | El CR necesitó una revisión manual y fue rechazado. (The CR needed a manual review and was rejected) |
  | **direct_debit_rejected**: (+-=CR.recovery_status)        |       831 | El intento de débito directo SEPA falló. (Our last attempt of SEPA direct debit to charge the customer was rejected ) |
  | **transaction_declined**:                                 |        48 | No se pudo enviar el dinero al cliente. (We failed to send the funds to the customer) |
  | **canceled**:                                             |        33 | El usuario no confirmó el CR en la app, fue cancelado automáticamente. (The user didn't confirm the cash request in-app, we automatically canceled it) |
  | **En los datos proporcionados, NO aparecen los valores:** |           |                                                              |
  | **approved**:                                             |         0 | CR is a 'regular' one (= without fees) and was approved either automatically or manually. Funds will be sent aprox. 7 days after the creation. |
  | **money_sent**:                                           |         0 | We transferred the fund to the customer account. Will change to active once we detect that the user received the funds (using user's bank history). |
  | **pending**:                                              |         0 | The CR is pending a manual review from an analyst.           |
  | **waiting_user_confirmation**:                            |         0 | The user needs to confirm in-app that he want the CR (for legal reasons) |
  | **waiting_reimbursement**:                                |         0 | We were not able to estimate a date of reimbursement, the user needs to choose one in the app. |
     

---
---
#### Columnas que figuran en la documentacion per que NO TENEMOS EN LA BASE DE DATOS:

* **(CR.reason)**:
  Filled only if the CR was manually reviewed and rejected. **That's the rejection's reason displayed in-app**. 

* **(CR.cash_request_debited_date)**:
  Filled only if a SEPA direct debit was sent. It's the date were the latest direct debit was seen on the user account.


### Fees (FE)

* **FE.id**:
  Unique ID of the fee object.

* **FE.cash_request_id**:
  Unique ID of the CR linked to this fee.

* **FE.total_amount**:
  Amount of the fee (including VAT). **TODO: Es un importe o un porcentaje ??**

* **FE.reason**:
  Description of the fee.

* **FE.created_at**:
  Timestamp of the fee's creation.

* **FE.updated_at**:
  Timestamp of the latest fee's details update.

* **FE.paid_at**:
  Timestamp of the fee's payment.

* **FE.from_date**:
  Apply only to postpone fees. Initial date of reimbursement for the CR.

* **FE.to_date**:
  Apply only to postpone fees. New date of reimbursement for the CR.


* **FE.type**:
  Type of fee. Possible values are:

  | Campo                | Registros | Descripcion                                                  |
  | -------------------- | --------: | ------------------------------------------------------------ |
  | **instant_payment**: |     11095 | Fees por adelanto instantáneo. (fees for instant cash request (send directly after user's request, **through SEPA Instant Payment**) ) |
  | **postpone**:        |      7766 | Fees por la solicitud de posponer un reembolso. (fees created when a user want to postpone the reimbursment of a CR) |
  | **incident**:        |      2196 | Fees por fallos de reembolsos. (fees for failed reimbursement. Created after a failed direct debit) |
  | **split_payment**:   |         0 | Fees por pago fraccionado (en caso de un incidente).<br/>(futures fees for split payment (in case of an incident, we'll soon offer the possibility to our users to reimburse in multiples installements)) |


* **FE.status**:
  (= does the fees was successfully charged). Possibles values are:

  | Campo          | Registros | Descripcion                                                  |
  | -------------- | --------: | ------------------------------------------------------------ |
  | **accepted**:  |     14841 | El fee fue cobrado exitosamente. (fees were successfully charged) |
  | **cancelled**: |      4934 | El fee fue **creado pero cancelado** por algún motivo. <br>Se utiliza para solucionar problemas con las `fees`, pero se refiere principalmente a `fees` `postpone` que fallaron. Estamos cobrando las `fees` en el momento de la solicitud del `postpone`. <br/ >Si falla, no se acepta el `postpone` y la fecha de reembolso sigue siendo la misma. |
  |                |           | (Fee was created and cancelled for some reasons. It's used to fix issues with fees but it mainly concern postpone fees who failed. We are charging the fees at the moment of the postpone request. If it failed, the postpone is not accepted and the reimbursement date still the same.) |
  | **rejected**:  |      1194 | El último intento de cobrar el fee falló. (the last attempt to charge the fee failed.) |
  | **confirmed**: |        88 | El usuario completó una acción que creó un fee. (the user made an action who created a fee. <br/>It will normally get charged at the moment of the CR's reimbursement. In some rare cases, postpones are confirmed without being charges due to a commercial offer.) |


* **FE.category**:
  Describe the reason of the incident fee. 2 possibles values:

  | Campo                       | Registros | Descripcion                                                  |
  | --------------------------- | --------: | ------------------------------------------------------------ |
  | **rejected_direct_debit**:  |     18861 | Fees creados cuando el banco del usuario rechaza el primer débito directo. (fees created when user's bank rejects the first direct debit) |
  | **month_delay_on_payment**: |      1599 | Fees creados cada mes hasta que el incidente se cierre. (fees created every month until the incident is closed) |
  | **NULL**:                   |       597 | ?? No figura a la documentacio                               |


* **FE.charge_moment**:
  When the fee will be charge. 2 possibles values:

  | Campo       | Registros | Descripcion                                                  |
  | ----------- | --------: | ------------------------------------------------------------ |
  | **after**:  |     16720 | El fee se cobra cuando el CR es reembolsado. (the fee should be charged at the moment of the CR's reimbursement) |
  | **before**: |      4337 | El fee se cobra en el momento de su creación. (the fee should be charged at the moment of its creation) |

## EDA

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm

cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
df_jo = pm.df('df_jo')

#df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = pm.sort("df_jo", ['created_at','created_at_fe']).reset_index()
df_jo = df_jo.drop(columns=['index'])

#df_jo = df_jo.drop(columns=['Mes_created_at'])
df_jo_cp = df_jo.copy()
df_jo_cp['cr_received_date'] = df_jo_cp.cash_request_received_date
#df_jo.info()
pd.options.display.max_columns = None
#display(df_jo)

In [ ]:
df_jo = pm.df('df_jo')
#df_jo['created_at'] = df_jo['created_at'].dt.to_period('d')
#df_jo['created_at'] = pd.to_datetime(df_jo['created_at']).dt.date
#df_jo['created_at'] = df_jo['created_at'].dt.to_period('m').dt.to_timestamp()
#pm.format_to_dates(df_jo, time_format='d') # 'min','s'
#df_jo.info()
#display(df_jo)

### Estudio de registros

In [11]:
counts = cr_cp.status.value_counts()
display(counts)

counts = cr_cp.transfer_type.value_counts()
display(counts)

counts = cr_cp.recovery_status.value_counts()
display(counts)

status
money_back               16395
rejected                  6568
direct_debit_rejected      831
active                      59
transaction_declined        48
direct_debit_sent           34
canceled                    33
Name: count, dtype: int64

transfer_type
instant    13882
regular    10086
Name: count, dtype: int64

recovery_status
nice                    20639
completed                2467
pending                   845
pending_direct_debit       16
cancelled                   1
Name: count, dtype: int64

In [4]:
counts = fe_cp.status.value_counts()
display(counts)

counts = fe_cp.type.value_counts()
display(counts)

counts = fe_cp.category.value_counts()
display(counts)

counts = fe_cp.charge_moment.value_counts()
display(counts)


status
accepted     14841
cancelled     4934
rejected      1194
confirmed       88
Name: count, dtype: int64

type
instant_payment    11095
postpone            7766
incident            2196
Name: count, dtype: int64

category
nice                      18861
rejected_direct_debit      1599
month_delay_on_payment      597
Name: count, dtype: int64

charge_moment
after     16720
before     4337
Name: count, dtype: int64

In [12]:
pd.options.display.max_columns = None
good_cr = ['approved', 'money_sent', 'pending', 'direct_debit_sent', 'active', 'money_back']
good_fe = ['confirmed', 'accepted', 'cr_regular']
df_jo['needs_m_check2'] = (~((df_jo['stat_cr'].isin(good_cr)) & (df_jo['stat_fe'].isin(good_fe)))).astype(int)
fields.append('needs_m_check2')
display(df_jo[fields].head(5))
#display(df_jo.head(5))

,id_cr,user_id,created_at,transfer_type,type,stat_cr,amount,fee,n_fees,n_backs,needs_m_check,n_recovery,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,recovery_status,needs_m_check2
0,5,804,2019-12-10 19:05:21.596873,regular,nice,rejected,100.0,NaN,0,0,1,0,cr_regular,0,NaT,NaT,NaN,NaT,2020-01-09 19:05:21.596363,29 days 23:59:59.999490,NaT,NaT,NaN,nice,1
1,70,231,2019-12-10 19:50:12.347780,regular,nice,rejected,100.0,NaN,0,0,1,0,cr_regular,0,NaT,NaT,NaN,NaT,2020-01-09 19:50:12.347780,30 days 00:00:00,NaT,NaT,NaN,nice,1
2,7,191,2019-12-10 19:13:35.825460,regular,nice,rejected,100.0,NaN,0,0,1,0,cr_regular,0,NaT,NaT,NaN,NaT,2020-01-09 19:13:35.825041,29 days 23:59:59.999581,NaT,NaT,NaN,nice,1
3,10,761,2019-12-10 19:16:10.880172,regular,nice,rejected,99.0,NaN,0,0,1,0,cr_regular,0,NaT,NaT,NaN,NaT,2020-01-09 19:16:10.879606,29 days 23:59:59.999434,NaT,NaT,NaN,nice,1
4,1594,7686,2020-05-06 09:59:38.877376,regular,nice,rejected,100.0,NaN,0,0,1,0,cr_regular,0,NaT,NaT,NaN,NaT,2020-06-05 22:00:00.000000,30 days 12:00:21.122624,NaT,NaT,NaN,nice,1


### Formato de fechas

In [31]:
# df = df_jo.copy()
# timeFormat ='d' #s
# df['created_at'] = df['created_at'].dt.to_period(timeFormat) #'Min')
# df['created_at_fe'] = df['created_at_fe'].dt.to_period(timeFormat) #'Min')
# df['updated_at'] = df['updated_at'].dt.to_period(timeFormat) #'Min')
# df['updated_at_fe'] = df['updated_at_fe'].dt.to_period(timeFormat) #'Min')

# df['to_receive_ini'] = pd.to_timedelta(df['to_receive_ini']).round(timeFormat)
# #df['to_receive_ini'] = df['to_receive_ini'].timedelta(seconds=math.ceil(df['to_receive_ini'].total_seconds()))

# df['to_receive_bank'] = pd.to_timedelta(df['to_receive_bank']).round(timeFormat)
# df['to_reimbur'] = pd.to_timedelta(df['to_reimbur']).round(timeFormat)
# df['to_reimbur_cash'] = pd.to_timedelta(df['to_reimbur_cash']).round(timeFormat)
# df['to_end'] = pd.to_timedelta(df['to_end']).round(timeFormat)
# df['to_send'] = pd.to_timedelta(df['to_send']).round(timeFormat)

# df['money_back_date'] = df['money_back_date'].dt.to_period(timeFormat)
# df['send_at'] = df['send_at'].dt.to_period(timeFormat)
# df['paid_at'] = df['paid_at'].dt.to_period(timeFormat)
# df['moderated_at'] = df['moderated_at'].dt.to_period(timeFormat)
# df['from_date'] = df['from_date'].dt.to_period(timeFormat)
# df['to_date'] = df['to_date'].dt.to_period(timeFormat)

# df['reco_creation'] = df['reco_creation'].dt.to_period(timeFormat)
# df['reco_last_update'] = df['reco_last_update'].dt.to_period(timeFormat)
# display(df.head(5))

### cohort_analysis_2

In [6]:
cohort_analysis_2 = (
    df_jo.query('stat_fe == "accepted" | stat_cr == "money_back"')
        .groupby(['user_id', 'Mes_created_at'], as_index=False)
        .agg(
            total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),            
            # Contar los valores únicos de 'stat_cr' donde su valor sea 'money_back'
            total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),
            # Contar los valores únicos de 'id_cr' donde 'stat_cr' es igual a 'money_back'
            Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())
        )
)


In [13]:
# Restablecer el índice para un DataFrame limpio (opcional, ya garantizado por as_index=False)
cohort_analysis_2.reset_index(drop=True, inplace=True)

# Calcular el índice como porcentaje entre 'total_paid_fees' y 'total_paid_cr'
cohort_analysis_2['cash_index'] = (
    cohort_analysis_2['total_paid_fees'] / cohort_analysis_2['total_paid_cr'] ) * 100

# Reemplazar valores 'inf' con 0 para manejar divisiones por cero
cohort_analysis_2['cash_index'] = cohort_analysis_2['cash_index'].replace(np.inf, 0)

# Calcular la fecha del último pedido por usuario a partir del DataFrame original
df_jo['created_at'] = pd.to_datetime(df_jo['created_at'])  # Asegurarse de que el formato sea datetime
last_order_per_user = (
    df_jo.groupby('user_id')['created_at']
    .max()  # Obtener la fecha más reciente de pedido para cada usuario
    .dt.to_period('M')  # Convertir a periodo mensual
    .reset_index()  # Restablecer el índice para facilitar el merge
)

# Incorporar la fecha del último pedido en el DataFrame de análisis de cohortes
cohort_analysis_2 = pd.merge(
    cohort_analysis_2,
    last_order_per_user.rename(columns={'created_at': 'last_order'}),
    on='user_id',
    how='left'
)

display(cohort_analysis_2.head(5))

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,cash_index,last_order
0,47,2020-04,0.0,100.0,1,0.000000,2020-10
1,47,2020-05,5.0,10.0,1,50.000000,2020-10
2,47,2020-08,10.0,10.0,1,100.000000,2020-10
3,47,2020-09,5.0,5.0,1,100.000000,2020-10
4,47,2020-10,10.0,6.0,2,166.666667,2020-10


### Estudio de casos de clientes concretos (user_id)

* 102105
* 16391 
* 2002
* 13851
* 1987 
* ...
* 16391 # 2002, 1987, 13851, 16391, 102105

* user_ids = [13851] [2002] , 1987, 1946, 90, 526, 12934] #, 12274 54879 12441, 13851, 16391, 430,  63894,18730,10116,21465, 99000262]
* vips 12934 526
* 90 Este se esta gestionando mal: todos instant, con demoras y sin gestion por 
* 1946 Parece un ejemplo de buena gestion, al final tiene un instant y se le ha dado margen el las demoras.
* 1987 Parece un ejemplo de buen usuario, se pasa a instant para siempre.

Casos que esta examinando Alba:
* 19655
* 21465
* 14631


In [14]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 102105])

display(cohort_analysis_2[cohort_analysis_2.user_id == 16391])
display(cohort_analysis_2[cohort_analysis_2.user_id == 2002])
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851])
display(cohort_analysis_2[cohort_analysis_2.user_id == 1987]) #.tail(60))

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,cash_index,last_order
15482,102105,2020-10,0.0,100.0,1,0.0,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,cash_index,last_order
5421,16391,2020-06,5.0,100.0,1,5.0,2020-10
5422,16391,2020-08,10.0,100.0,1,10.0,2020-10
5423,16391,2020-10,5.0,100.0,1,5.0,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,cash_index,last_order
747,2002,2020-01,0.0,100.0,1,0.0,2020-10
748,2002,2020-02,0.0,100.0,1,0.0,2020-10
749,2002,2020-03,0.0,100.0,1,0.0,2020-10
750,2002,2020-04,0.0,100.0,1,0.0,2020-10
751,2002,2020-05,0.0,100.0,1,0.0,2020-10
752,2002,2020-06,0.0,100.0,1,0.0,2020-10
753,2002,2020-07,15.0,100.0,1,15.0,2020-10
754,2002,2020-08,0.0,100.0,1,0.0,2020-10
755,2002,2020-09,0.0,100.0,1,0.0,2020-10
756,2002,2020-10,5.0,100.0,1,5.0,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,cash_index,last_order
4640,13851,2020-08,5.0,150.0,2,3.333333,2020-10
4641,13851,2020-10,5.0,100.0,1,5.000000,2020-10


,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,cash_index,last_order
733,1987,2019-12,0.0,100.0,1,0.0,2020-10
734,1987,2020-01,0.0,100.0,1,0.0,2020-10
735,1987,2020-02,0.0,100.0,1,0.0,2020-10
736,1987,2020-03,0.0,100.0,1,0.0,2020-10
737,1987,2020-04,0.0,100.0,1,0.0,2020-10
738,1987,2020-05,0.0,100.0,1,0.0,2020-10
739,1987,2020-06,10.0,100.0,1,10.0,2020-10
740,1987,2020-08,15.0,100.0,1,15.0,2020-10
741,1987,2020-10,10.0,100.0,2,10.0,2020-10


In [15]:
user_id = 2002# 16391 # 2002, 1987, 13851, 16391, 102105
display(cohort_analysis_2[cohort_analysis_2.user_id == user_id])

#print("Casos segun Cash Request ID")
pd.options.display.max_columns = None
for id in ([-8177]): # 16391 20108, 20104, 20112,
    df_t = df_jo[df_jo['id_cr'] == id].sort_values(['created_at','created_at_fe']).reset_index()
    print(f"Cash Request ID: {id}")
    display(df_t[fields])

user_ids = [user_id] 
pd.options.display.max_columns = None
#print("Casos segun Cash User ID")
for id in (user_ids):
    df_t = df_jo[(df_jo['user_id'] == id)]#.reset_index()
    df_t = df_t[df_t['stat_cr'] == 'money_back']
    df_t = df_t[df_t['stat_fe'] == 'accepted']
    
    df_t = df_t.sort_values(['created_at','created_at_fe']).reset_index(drop=True)
    #df_t.set_index('id_cr', inplace=True)
    print(f"Only money_back - user_id {id}")
    display(df_t[fields])
    df_t = df_jo[(df_jo['user_id'] == id) ].sort_values(['created_at','created_at_fe']).reset_index(drop=True)

    print(f"user_id {id}")
    display(df_t[fields])

,user_id,Mes_created_at,total_paid_fees,total_paid_cr,Num_Solicitudes,cash_index,last_order
747,2002,2020-01,0.0,100.0,1,0.0,2020-10
748,2002,2020-02,0.0,100.0,1,0.0,2020-10
749,2002,2020-03,0.0,100.0,1,0.0,2020-10
750,2002,2020-04,0.0,100.0,1,0.0,2020-10
751,2002,2020-05,0.0,100.0,1,0.0,2020-10
752,2002,2020-06,0.0,100.0,1,0.0,2020-10
753,2002,2020-07,15.0,100.0,1,15.0,2020-10
754,2002,2020-08,0.0,100.0,1,0.0,2020-10
755,2002,2020-09,0.0,100.0,1,0.0,2020-10
756,2002,2020-10,5.0,100.0,1,5.0,2020-10


Cash Request ID: -8177


,id_cr,user_id,created_at,transfer_type,type,stat_cr,amount,fee,n_fees,n_backs,needs_m_check,n_recovery,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,recovery_status,needs_m_check2


Only money_back - user_id 2002


,id_cr,user_id,created_at,transfer_type,type,stat_cr,amount,fee,n_fees,n_backs,needs_m_check,n_recovery,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,recovery_status,needs_m_check2
0,8177,2002,2020-07-15 13:17:11.174285,regular,postpone,money_back,100.0,5.0,1,8,0,0,accepted,1749,2020-07-22 01:33:56.400884,2020-10-13 14:25:00.882490,Postpone Cash Request 8177,2020-08-03 09:01:41.363548,2020-08-05 01:33:48,20 days 12:16:36.825715,2020-08-04 22:00:00.000,2020-08-15 01:33:48.128,before,nice,0
1,8177,2002,2020-07-15 13:17:11.174285,regular,postpone,money_back,100.0,5.0,2,9,0,0,accepted,1839,2020-07-23 11:31:44.836318,2020-10-13 14:25:02.005106,Postpone Cash Request 8177,2020-08-03 09:01:41.363548,2020-08-05 01:33:48,20 days 12:16:36.825715,2020-08-15 01:33:48.128,2020-08-30 01:33:48.128,before,nice,0
2,8177,2002,2020-07-15 13:17:11.174285,regular,postpone,money_back,100.0,5.0,3,10,0,0,accepted,1977,2020-07-25 19:20:00.560197,2020-10-13 14:25:04.227093,Postpone Cash Request 8177,2020-08-03 09:01:41.363548,2020-08-05 01:33:48,20 days 12:16:36.825715,2020-08-30 01:33:48.128,2020-09-14 01:33:48.128,before,nice,0
3,24248,2002,2020-10-25 07:41:00.266568,instant,instant_payment,money_back,100.0,5.0,4,13,0,0,accepted,17289,2020-10-25 07:41:49.198215,2020-10-25 07:41:49.198245,Instant Payment Cash Request 24248,2020-11-09 19:11:48.425712,2020-11-09 22:00:00,15 days 14:18:59.733432,NaT,NaT,after,nice,0


user_id 2002


,id_cr,user_id,created_at,transfer_type,type,stat_cr,amount,fee,n_fees,n_backs,needs_m_check,n_recovery,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,recovery_status,needs_m_check2
0,423,2002,2020-01-10 10:55:20.757139,regular,nice,money_back,100.0,NaN,0,1,0,0,cr_regular,0,NaT,NaT,NaN,2020-02-06 23:00:00.000000,2020-02-06 23:00:00,27 days 12:04:39.242861,NaT,NaT,NaN,nice,0
1,697,2002,2020-02-18 16:36:51.249037,regular,nice,money_back,100.0,NaN,0,2,0,0,cr_regular,0,NaT,NaT,NaN,2020-02-27 23:00:00.000000,2020-02-27 23:00:00,9 days 06:23:08.750963,NaT,NaT,NaN,nice,0
2,835,2002,2020-03-10 07:47:39.337041,regular,nice,money_back,100.0,NaN,0,3,0,0,cr_regular,0,NaT,NaT,NaN,2020-04-14 20:25:59.132327,2020-04-05 22:00:00,26 days 14:12:20.662959,NaT,NaT,NaN,nice,0
3,1172,2002,2020-04-14 21:03:09.519326,regular,nice,money_back,100.0,NaN,0,4,0,0,cr_regular,0,NaT,NaT,NaN,2020-05-14 21:05:09.488707,2020-05-05 22:00:00,21 days 00:56:50.480674,NaT,NaT,NaN,nice,0
4,1800,2002,2020-05-15 04:09:51.091889,regular,nice,money_back,100.0,NaN,0,5,0,0,cr_regular,0,NaT,NaT,NaN,2020-06-10 22:17:53.131912,2020-06-06 22:00:00,22 days 17:50:08.908111,NaT,NaT,NaN,nice,0
5,2949,2002,2020-06-10 23:34:13.556501,regular,nice,money_back,100.0,NaN,0,6,0,0,cr_regular,0,NaT,NaT,NaN,2020-07-07 22:00:00.000000,2020-07-06 22:00:00,25 days 22:25:46.443499,NaT,NaT,NaN,nice,0
6,8177,2002,2020-07-15 13:17:11.174285,regular,postpone,money_back,100.0,5.0,0,7,1,0,cancelled,1674,2020-07-20 20:43:33.629841,2020-10-13 14:25:15.265408,Postpone Cash Request 8177,2020-08-03 09:01:41.363548,2020-08-05 01:33:48,20 days 12:16:36.825715,2020-08-04 22:00:00.000,2020-08-19 22:00:00.000,after,nice,1
7,8177,2002,2020-07-15 13:17:11.174285,regular,postpone,money_back,100.0,5.0,1,8,0,0,accepted,1749,2020-07-22 01:33:56.400884,2020-10-13 14:25:00.882490,Postpone Cash Request 8177,2020-08-03 09:01:41.363548,2020-08-05 01:33:48,20 days 12:16:36.825715,2020-08-04 22:00:00.000,2020-08-15 01:33:48.128,before,nice,0
8,8177,2002,2020-07-15 13:17:11.174285,regular,postpone,money_back,100.0,5.0,2,9,0,0,accepted,1839,2020-07-23 11:31:44.836318,2020-10-13 14:25:02.005106,Postpone Cash Request 8177,2020-08-03 09:01:41.363548,2020-08-05 01:33:48,20 days 12:16:36.825715,2020-08-15 01:33:48.128,2020-08-30 01:33:48.128,before,nice,0
9,8177,2002,2020-07-15 13:17:11.174285,regular,postpone,money_back,100.0,5.0,3,10,0,0,accepted,1977,2020-07-25 19:20:00.560197,2020-10-13 14:25:04.227093,Postpone Cash Request 8177,2020-08-03 09:01:41.363548,2020-08-05 01:33:48,20 days 12:16:36.825715,2020-08-30 01:33:48.128,2020-09-14 01:33:48.128,before,nice,0


### Estudio 2 de clientes

In [16]:
# fields = ['id_cr','created_at','transfer_type','type','stat_cr' ,'amount','fee','n_fees','n_backs', # 'good_user',
#           'stat_fe','id_fe','created_at_fe','updated_at_fe','reason','money_back_date', 'reimbursement_date',
#           'to_reimbur','from_date','to_date', 'charge_moment' # 'paid_at', 'to_end',, #,'user_id', 'cr_received_date','recovery_status'
#           #'to_receive_ini','to_receive_bank' #,'to_reimbur_cash', 'updated_at', 'to_send','send_at','moderated_at'
# ]

df_jo = df_jo.reindex(columns=fields)
df_jo.reset_index(drop=True, inplace=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
#display(df.head(5))
display(df_jo[df_jo['user_id'] == 2002]) #.reset_index()

,id_cr,user_id,created_at,transfer_type,type,stat_cr,amount,fee,n_fees,n_backs,needs_m_check,n_recovery,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,recovery_status,needs_m_check2
976,1800,2002,2020-05-15 04:09:51.091889,regular,nice,money_back,100.0,NaN,0,5,0,0,cr_regular,0,NaT,NaT,NaN,2020-06-10 22:17:53.131912,2020-06-06 22:00:00,22 days 17:50:08.908111,NaT,NaT,NaN,nice,0
6920,423,2002,2020-01-10 10:55:20.757139,regular,nice,money_back,100.0,NaN,0,1,0,0,cr_regular,0,NaT,NaT,NaN,2020-02-06 23:00:00.000000,2020-02-06 23:00:00,27 days 12:04:39.242861,NaT,NaT,NaN,nice,0
7486,1172,2002,2020-04-14 21:03:09.519326,regular,nice,money_back,100.0,NaN,0,4,0,0,cr_regular,0,NaT,NaT,NaN,2020-05-14 21:05:09.488707,2020-05-05 22:00:00,21 days 00:56:50.480674,NaT,NaT,NaN,nice,0
9349,835,2002,2020-03-10 07:47:39.337041,regular,nice,money_back,100.0,NaN,0,3,0,0,cr_regular,0,NaT,NaT,NaN,2020-04-14 20:25:59.132327,2020-04-05 22:00:00,26 days 14:12:20.662959,NaT,NaT,NaN,nice,0
10605,697,2002,2020-02-18 16:36:51.249037,regular,nice,money_back,100.0,NaN,0,2,0,0,cr_regular,0,NaT,NaT,NaN,2020-02-27 23:00:00.000000,2020-02-27 23:00:00,9 days 06:23:08.750963,NaT,NaT,NaN,nice,0
13562,2949,2002,2020-06-10 23:34:13.556501,regular,nice,money_back,100.0,NaN,0,6,0,0,cr_regular,0,NaT,NaT,NaN,2020-07-07 22:00:00.000000,2020-07-06 22:00:00,25 days 22:25:46.443499,NaT,NaT,NaN,nice,0
14316,8177,2002,2020-07-15 13:17:11.174285,regular,postpone,money_back,100.0,5.0,1,8,0,0,accepted,1749,2020-07-22 01:33:56.400884,2020-10-13 14:25:00.882490,Postpone Cash Request 8177,2020-08-03 09:01:41.363548,2020-08-05 01:33:48,20 days 12:16:36.825715,2020-08-04 22:00:00.000,2020-08-15 01:33:48.128,before,nice,0
14317,8177,2002,2020-07-15 13:17:11.174285,regular,postpone,money_back,100.0,5.0,2,9,0,0,accepted,1839,2020-07-23 11:31:44.836318,2020-10-13 14:25:02.005106,Postpone Cash Request 8177,2020-08-03 09:01:41.363548,2020-08-05 01:33:48,20 days 12:16:36.825715,2020-08-15 01:33:48.128,2020-08-30 01:33:48.128,before,nice,0
14318,8177,2002,2020-07-15 13:17:11.174285,regular,postpone,money_back,100.0,5.0,3,10,0,0,accepted,1977,2020-07-25 19:20:00.560197,2020-10-13 14:25:04.227093,Postpone Cash Request 8177,2020-08-03 09:01:41.363548,2020-08-05 01:33:48,20 days 12:16:36.825715,2020-08-30 01:33:48.128,2020-09-14 01:33:48.128,before,nice,0
14319,8177,2002,2020-07-15 13:17:11.174285,regular,postpone,money_back,100.0,5.0,0,7,1,0,cancelled,1674,2020-07-20 20:43:33.629841,2020-10-13 14:25:15.265408,Postpone Cash Request 8177,2020-08-03 09:01:41.363548,2020-08-05 01:33:48,20 days 12:16:36.825715,2020-08-04 22:00:00.000,2020-08-19 22:00:00.000,after,nice,1


### Estudio de top FEES

In [17]:
pd.options.display.max_rows = None
tops = df_jo[df_jo['stat_cr' ]== 'money_back'].groupby('user_id').agg(fees=('fee','sum'))
tops = df_jo[df_jo['stat_fe'] == 'accepted'  ].groupby('user_id').agg(fees=('fee','sum'))
#display(df_jo[tops])
top_users = tops.sort_values(by='fees', ascending=False).iloc[:10].reset_index()
display(top_users)
#top_users = tops.sort_values(by='fees', ascending=True).iloc[:10]
#display(top_users)

#display(df_jo[top_users])

,user_id,fees
0,17144,75.0
1,12934,55.0
2,4982,35.0
3,99021532,35.0
4,4636,35.0
5,5189,35.0
6,4317,35.0
7,13404,35.0
8,9199,35.0
9,17603,35.0
